In [11]:
import torch
import sys
import os.path as osp
import os
import sys
import numpy as np

sys.path.append("/afs/cern.ch/work/m/mgarciam/private/Tracking_wcoc/")
from src.dataset.dataset import SimpleIterDataset
from src.utils.train_utils import to_filelist
from torch.utils.data import DataLoader
#import dgl  # CPU only version for now
from tqdm import tqdm
from torch_scatter import scatter_sum
import matplotlib.pyplot as plt
import pickle
import numpy as np
import mplhep as hep


hep.style.use("CMS")
import matplotlib
matplotlib.rc('font', size=13)


/eos/experiment/fcc/ee/datasets/DC_tracking/Pythia/gun_fakeCalo_g1/reco_gun_1000.root

In [12]:
# This block is the same as 1_dataset.ipynb

#"/eos/experiment/fcc/ee/datasets/DC_tracking/Pythia_evaluation/Zcard/reco_Zcard_1.root"
datasets = {
    "test": "/eos/experiment/fcc/ee/datasets/DC_tracking/Pythia/scratch/Zcard_CLD_background/3/reco_2.root",
    "train": "/eos/experiment/fcc/ee/datasets/DC_tracking/Pythia/scratch/Zcard_CLD_background/3/reco_2.root",
}

class Args:
    def __init__(self, datasets):
        self.data_train = [datasets]
        self.data_val = [datasets]
        #self.data_train = files_train
        self.data_config = '/afs/cern.ch/work/m/mgarciam/private/Tracking_wcoc/config_files/config_tracking_global_vector_overlay.yaml'
        self.extra_selection = None
        self.train_val_split = 0.8
        self.data_fraction = 1
        self.file_fraction = 1
        self.fetch_by_files = False
        self.fetch_step = 1
        self.steps_per_epoch = None
        self.in_memory = False
        self.local_rank = None
        self.copy_inputs = False
        self.no_remake_weights = False
        self.batch_size = 2
        self.num_workers = 0
        self.demo = False
        self.laplace = False
        self.diffs = False
        self.class_edges = False


args = {key: Args(value) for key, value in datasets.items()}

datas = {}
files_dict = {}
for key in datasets:
    train_range = (0, args[key].train_val_split)
    train_file_dict, train_files = to_filelist(args[key], 'val')
    train_data = SimpleIterDataset(train_file_dict, args[key].data_config, for_training=False,
                                   extra_selection=args[key].extra_selection,
                                   remake_weights=True,
                                   load_range_and_fraction=(train_range, args[key].data_fraction),
                                   file_fraction=args[key].file_fraction,
                                   fetch_by_files=args[key].fetch_by_files,
                                   fetch_step=args[key].fetch_step,
                                   infinity_mode=False,
                                   in_memory=args[key].in_memory,
                                   async_load=False,
                                   name='train')
    datas[key] = train_data
    files_dict[key] = train_files

None 1
None 1


In [13]:
import plotly
import plotly.graph_objects as go
import plotly.offline as pyo
from plotly.subplots import make_subplots

In [14]:
itera = iter(train_data)

=== Restarting DataIter train, seed=None ===


In [15]:
# X1 =next(itera) 
g, y = next(itera) 



hit_particle_link tensor([  0.,  41.,  43.,  47.,  48.,  49.,  50.,  51.,  54.,  63.,  64.,  66.,
         67.,  69.,  70.,  78.,  79.,  84.,  85., 144., 145., 146., 150., 151.,
        152., 153., 154., 155., 156., 164., 165., 166., 167., 168., 169., 170.,
        171., 175., 176., 177., 178., 179., 204., 205., 206., 207., 208., 211.,
        212., 218., 231., 232., 233., 234., 235., 236., 320., 321., 322., 323.,
        324., 325., 330., 331., 332., 336., 340., 343., 345., 366., 369., 370.,
        382., 383., 384., 385., 386., 387., 400., 401., 402., 403., 414., 415.,
        416., 417., 418., 428., 429., 442., 443., 510., 511., 512., 513., 514.,
        516., 517., 518., 520., 591., 592., 593., 594., 595., 597., 598., 599.,
        600., 601., 602., 603., 604., 605., 606., 607., 608., 609., 610., 611.,
        612., 613., 614., 615., 616., 617., 618., 619., 620., 621., 622., 624.,
        625., 626., 627., 628., 629., 630., 631., 642., 643., 644., 646., 647.,
        648., 649., 65

/afs/cern.ch/work/m/mgarciam/private/miniconda/miniconda3/envs/graphgps0/lib/python3.10/site-packages/dgl/heterograph.py:72: DGLWarning:

Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.



In [19]:
weight_not_b = len(g.ndata["is_overlay"])/(2*torch.sum(g.ndata["is_overlay"]==0))
weight_b = len(g.ndata["is_overlay"])/(2*torch.sum(g.ndata["is_overlay"]==1))
weight_not_b, weight_b

(tensor(4.7446), tensor(0.5589))

In [16]:

import pandas as pd
import plotly.express as px
from src.logger.plotting_tools import shuffle_truth_colors

mask = (g.ndata['particle_number'] !=-1) 
tidx =  1*(g.ndata['particle_number'][mask]==0).view(-1,1)
#tidx =    1*(g.ndata['hit_link_modified'][mask].view(-1,1))+1
features =  (torch.ones_like(g.ndata['particle_number'][mask].view(-1,1)))
cellid= g.ndata['particle_number'][mask].view(-1,1)
X = g.ndata["pos_hits_xyz"][mask] #[mask]
data = {
            "X":X[:, 0].view(-1, 1).detach().cpu().numpy(),
            "Y": X[:, 1].view(-1, 1).detach().cpu().numpy(),
            "Z": X[:, 2].view(-1, 1).detach().cpu().numpy(),
            "tIdx": tidx.view(-1, 1).detach().cpu().numpy(),
            "features": features.view(-1, 1).detach().cpu().numpy(),
            "cellid":cellid.view(-1, 1).detach().cpu().numpy(),
        }
hoverdict = {}
# if hoverfeat is not None:
#     for j in range(hoverfeat.shape[1]):
#         hoverdict["f_" + str(j)] = hoverfeat[:, j : j + 1]
#     data.update(hoverdict)

# if nidx is not None:
#     data.update({"av_same": av_same})

df = pd.DataFrame(
np.concatenate([data[k] for k in data], axis=1),
columns=[k for k in data],
)
rdst = np.random.RandomState(1234567890)  # all the same
# shuffle_truth_colors(df, "tIdx", rdst)

hover_data = ["tIdx", "cellid"] #+ [k for k in hoverdict.keys()]
# if nidx is not None:
#     hover_data.append("av_same")
fig = px.scatter_3d(
df,
x="X",
y="Y",
z="Z",
color="tIdx",
size="features",
hover_data=hover_data,
template="simple_white",
color_continuous_scale=px.colors.sequential.Rainbow,
range_color=[0,1],
)
fig.update_traces(marker=dict(line=dict(width=0)))
fig.update_traces(marker_size = 2)

here


In [27]:
from torch_scatter import scatter_add, scatter_sum, scatter_min, scatter_max
coord = g.ndata["pos_hits_xyz"]
cluster_id = g.ndata["particle_number"]
min_x = scatter_min(coord[:, 0], cluster_id.long() - 1)[0]
min_z = scatter_min(coord[:, 2], cluster_id.long() - 1)[0]
min_y = scatter_min(coord[:, 1], cluster_id.long() - 1)[0]
max_x = scatter_max(coord[:, 0], cluster_id.long() - 1)[0]
max_z = scatter_max(coord[:, 2], cluster_id.long() - 1)[0]
max_y = scatter_max(coord[:, 1], cluster_id.long() - 1)[0]
diff_x = torch.abs(max_x - min_x)
diff_z = torch.abs(max_z - min_z)
diff_y = torch.abs(max_y - min_y)
mask_x = diff_x > 2000
mask_z = diff_z > 2800
mask_y = diff_y > 2800
mask_p = mask_x + mask_z + mask_y
mask_p

tensor([False, False, False, False, False, False, False, False, False])

In [201]:
diff_x, diff_z,diff_y

(tensor([679.0754,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000]),
 tensor([729.3204,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000]),
 tensor([1783.1533,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000]))